Nepal Earthquake OSM Data Exploration
=====================================

In [2]:
require '/opt/epic-osm/epic-osm.rb'
analysis_window = './nepal-earthquake.yml'
o  = EpicOSM.new(analysis_window: analysis_window)
aw = o.analysis_window

Successfully initialized Analysis Window: Nepal Earthquake



#<AnalysisWindow:0x007f3da7987ab0 @bounding_box=#<BoundingBox:0x007f3da798ce20 @bbox_array=["80.0134", "26.3033", "88.2202", "30.5149"], @bottom_left=[80.0134, 26.3033], @top_right=[88.2202, 30.5149], @active=false>, @time_frame=#<TimeFrame:0x007f3da798f4e0 @start_date=2015-04-25 00:00:00 +0000, @end_date=2015-05-31 00:00:00 +0000, @active=true>, @max_area=1000000000000, @min_area=1, @title="Nepal Earthquake", @changeset_tags=nil>

## Visualize some changesets information!

In [3]:
cSets_per_4hours = aw.changesets_x_hour(step: 4);
nil

{:unit=>:hour, :step=>4, :constraints=>{}, :collection=>"changesets", :type=>DomainObject::Changeset}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 00:00:00 +0000, "$lt"=>2015-04-25 04:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 04:00:00 +0000, "$lt"=>2015-04-25 08:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 08:00:00 +0000, "$lt"=>2015-04-25 12:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 12:00:00 +0000, "$lt"=>2015-04-25 16:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 16:00:00 +0000, "$lt"=>2015-04-25 20:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 20:00:00 +0000, "$lt"=>2015-04-26 00:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-26 00:00:00 +0000, "$lt"=>2015-04-26 04:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-26 04:00:00 +0000, "$lt"=>2015-04-26 08:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-26 08:0

In [161]:
users_per_4hours = cSets_per_4hours.collect{|bucket| 
  {time: bucket[:start_date],
   all_users: bucket[:objects].collect{|x| x.user}.uniq,
   hot_users: bucket[:objects].select{|c| c.comment =~ /hot/}.collect{|x| x.user}.uniq
  }
}

counts_per_4_hours = users_per_4hours.collect{ |d| 
  { time:      d[:time],
    all_users: d[:all_users].count,
    hot_users: d[:hot_users].count
  }
}; nil

In [162]:
puts "All users: #{ users_per_4hours.collect{|x| x[:all_users] }.flatten.uniq.length }"
puts "HOT users: #{ users_per_4hours.collect{|x| x[:hot_users] }.flatten.uniq.length }"
nil

All users: 7068
HOT users: 6075


In [169]:
#Write out JSON to visualize:
require 'json'
File.write('json/user_counts_per_4hour.json', counts_per_4_hours.to_json)

14586

## Calculate Cumulative New Users?

In [202]:
all_users = Set.new
hot_users = Set.new
cumulative_users = []

cSets_per_4hours.each do |bucket|
  t_all_users = bucket[:objects].collect{|x| x.user}.uniq.to_set
  t_hot_users = bucket[:objects].select{|c| c.comment =~ /hot/}.collect{|x| x.user}.uniq.to_set
  
  
  #Unless THIS set of users is a subset of ALL PREVIOUS users... add the difference!
  unless t_all_users.subset?(all_users)
    all_users = all_users.merge(t_all_users - all_users)
  end
  
  unless t_hot_users.subset?(hot_users)
    hot_users = hot_users.merge(t_hot_users - hot_users)
  end
    
  cumulative_users << {time: bucket[:start_date], all_users: all_users.size, hot_users: hot_users.size}
  
end

cumulative_users[0..10].each do |u|
  puts "#{u[:time]}: #{u[:hot_users]}, #{u[:all_users]}"
end

File.write('json/cumulative_user_count.json', cumulative_users.to_json)
nil;

2015-04-25 00:00:00 +0000: 0, 0
2015-04-25 04:00:00 +0000: 0, 4
2015-04-25 08:00:00 +0000: 0, 11
2015-04-25 12:00:00 +0000: 54, 74
2015-04-25 16:00:00 +0000: 197, 227
2015-04-25 20:00:00 +0000: 277, 313
2015-04-26 00:00:00 +0000: 382, 425
2015-04-26 04:00:00 +0000: 489, 539
2015-04-26 08:00:00 +0000: 630, 688
2015-04-26 12:00:00 +0000: 815, 888
2015-04-26 16:00:00 +0000: 966, 1062


testing: {{x}}